# Significant aspects of model construction

* econ status
* bubbles
* spatiality
* mobility

Metrics upon which to assess these:
* cases over time
* deaths over time
* effective R at different points
* constraint-specific measures (e.g. cases over time per district in models with districts)

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib as mpl
import matplotlib.pyplot as plt
import os
import pylab as plt
from os.path import isfile, join
from glob import glob

import seaborn as sns
sns.set(style="darkgrid")

In [ ]:
def processModelOutput(filename):
    r_data = pd.read_csv(filename, sep='\t')
    r_data['districtId'] = [int(str(x).split('_')[1]) for x in r_data["myId"]]
    return {'results': r_data}

def findROFInstance():
    # total number of individuals infected by the introduction of an infectious person
    return beta * n

def WorldBankMASONmodel(seed, paramfile, beta):
    
    dataDir = "/home/wb488473/Disease-Modelling-SSA/data/"
    numDays = 30 * 6
    exportFilename = "exportMe" + seed + "_" + beta + ".txt"
    subprocess.call(['java', '-Xms20G', '-jar', '../java/WorldBankCovid19/libs/WB_covid19ABM.jar', str(numDays), dataDir, str(beta), seed, exportFilename])
    # Sophie upped the -Xms to 20G from 6GB
    return processModelOutput(exportFilename)

## Undifferentiated base case

Individuals are created without distinct statuses and their social bubbles are set to be the size of the entire community. There is a single district in which each agent is initialised and movement between districts (being meaningless) is zeroed out. Thus, status, social bubbles, and space are all excluded from this scenario.

The data directory associated with this case is in its own section, under the directory **data/verification/** and the specific set of parameters is as given in the file **data/verification/params_basecase.txt**. In particular:

* the census file has all economic statuses set to the value "Default" and all locations for home and work set to an imaginary "District 1"
* the daily transitions files are both set to 100% chance of moving from District 1 to District 1.
* mobility rates do not decrease after a lockdown...
* the lockdown file is empty of scheduled lockdowns, however, for belt-and-bracers insurance!

The infection parameters are the defaults as seen in the config/covasim_infect_transitions.txt

The files to vary are therefore the following:
* **numWeeklyInteractionsByStatus.txt**: a record of how many interactions agents with the "Default" status should have per week
* **lineList_10.txt**: the infections to seed in the single district.

An exploration of these is given below.

In [ ]:
# run the simulation with the given param file

#### Cases over time


In [ ]:
g = sns.relplot(x="time", y="scaled_sim_cases", kind="line", hue="beta", ci=None, data=nat_level, height=8, palette = "BuPu").set(title="Case projections of 10 beta runs - after scaling")
g.fig.autofmt_xdate()
g.savefig('../data/output/plots/sim_cases_scaled_10betaruns.png')

#### Deaths over time

#### Measuring R

## Introducing one constraint at a time

### Social Bubbles

### Status

### Spatial Partitioning

(districts exist)

## Combining these constraints

* Social bubbles + status
* Social bubbles + space
* Status + space
* All three

## Mobility

### Weekday/weekend

### Interdisctrict

## Future work: interventions

* lockdowns
* someday, hygiene
* someday, school closures

## Template

such description wow

#### Cases over time


#### Deaths over time

#### Measuring R

#### Constraint-specific?